In [0]:
CREATE OR REFRESH STREAMING TABLE bronze_orders
COMMENT 'Raw orders data ingested from the source system operational data'
TBLPROPERTIES ('quality' = 'bronze')
AS
SELECT *,
  _metadata.file_path AS input_file_path,
  current_timestamp AS ingestion_timestamp
FROM cloud_files ('/Volumes/circuitbox/landing/operational_data/orders/',
  'json',
  map('cloudFiles.inferColumnTypes', 'true')
  );

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_orders_clean
(
  CONSTRAINT valid_customer_id EXPECT (customer_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_order_id EXPECT (order_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_order_status EXPECT (order_status IN ('Pending','Shipped','Cancelled','Completed')),
  CONSTRAINT valid_payment_method EXPECT (payment_method IN ('Credit Card', 'Bank Transfer', 'PayPal'))
)
COMMENT 'Clean orders data'
TBLPROPERTIES ('quality' = 'silver')
AS 
SELECT
  order_id,
  customer_id,
  CAST(order_timestamp AS TIMESTAMP) AS order_timestamp,
  payment_method,
  items,
  order_status
  FROM STREAM(LIVE.bronze_orders);

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_orders
COMMENT 'Cleaned orders data'
TBLPROPERTIES ('quality' = 'silver')
AS 
SELECT
  order_id,
  customer_id,
  order_timestamp,
  payment_method,
  item.item_id,
  item.name as item_name,
  item.price as item_price,
  item.quantity as item_quantity,
  item.category as item_category,
  order_status
  FROM (SELECT  order_id,
                customer_id,
                order_timestamp,
                payment_method,
                order_status,
                explode(items) as item
                FROM STREAM(LIVE.silver_orders_clean));